En este colab hice las pruebas que terminaron en la clase miDataLoader() que se usa en la libreria dataLoader.py

In [134]:
import torch
import pandas as pd
import os
import numpy as np
from torch.utils.data import Dataset

In [9]:
Path_train = './csv/train_set.csv'
Path_val = './csv/validation_set.csv'
Path_test = './csv/test_set.csv'

True

In [53]:
np.random.seed(98) #fijo la semilla para repetibilidad del proyecto
train_df = pd.read_csv(Path_train, index_col= 0).sample(frac = 1) #cargo y mezlco el dataSet
train_df.head()

,M/F/I,artist,album,song,localPath,fileTrak,fileMel,sample/full
10786,instrumental,Panic! at the disco,instrumental,Folkin_ Around - Official Studio Instrumental(...,/data/instrumental/Panic! at the disco,Folkin_ Around - Official Studio Instrumental(...,Folkin_ Around - Official Studio Instrumental(...,sample
2077,hombre,Panic! At The Disco,Death of a Bachelor,08. Golden Days,/data/hombre/Panic! At The Disco/Death of a Ba...,08. Golden Days.mp3,08. Golden Days 14.pt,sample
32231,mujer,Taylor Swift,Taylor Swift - Lover (2019),08 - Paper Rings,/data/mujer/Taylor Swift/Taylor Swift - Lover ...,08 - Paper Rings.mp3,08 - Paper Rings 22.pt,sample
54,hombre,Imagine Dragons,Evolve,03 Whatever It Takes,/data/hombre/Imagine Dragons/Evolve,03 Whatever It Takes.mp3,03 Whatever It Takes 8.pt,sample
3579,hombre,Panic! At The Disco,"Too Weird To Live, Too Rare To Die! (Target Ex...",12. All The Boys (Bonus Track),/data/hombre/Panic! At The Disco/Too Weird To ...,12. All The Boys (Bonus Track).mp3,12. All The Boys (Bonus Track) 13.pt,sample


In [76]:
def one_hot_coder(df, col):
    """
    Esta funcion recibe un dataFrame y un string con el nombre de una columna del dataFrame.
    De modo que retorna un diccionario que transforma cada valor de la columna a un formato oneHot encoing
    """
    #obtengo los valores positbles de la columna objetivo
    keys = df[col].unique()
    #Genero un diccionario que transforma las keys en one hot
    key_to_oneHot = {}
    for i in range(len(keys)):
        one_hot_vector = [0] * len(keys)
        one_hot_vector[i] = 1
        key_to_oneHot [keys[i]] = one_hot_vector
    return key_to_oneHot


In [77]:
key_to_oneHot = one_hot_coder(train_df, 'M/F/I')
targets = list(train_df['M/F/I'])
targets = [key_to_oneHot[target] for target in targets]

print(train_df["M/F/I"][:5])
print('')
print(targets[:5])

10786    instrumental
2077           hombre
32231           mujer
54             hombre
3579           hombre
Name: M/F/I, dtype: object

[[1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0]]


In [78]:
def one_hot_decode(value, dictionary):
    """
    Se recibe un vector codificado en formato one hot y un diccionario el cual se uso para codificar el one hot
    De modo qu ese retorna la key <string> decodificada del vector recibido.
    """
    restored = 'NotFound'
    for key,dic_value in dictionary.items():
        if value == dic_value:
            restored = key
            break
    return restored

#test
a = [0,0,1]
one_hot_decode( a , key_to_oneHot)

'mujer'

In [ ]:
fPaths = '.' + samples['localPath'].str.strip() + '/' + samples['fileMel']
fPaths.iloc[5]

# Creando la clase Dataset

In [153]:
class MelDataset(Dataset):
    def __init__(self, df, path_col, file_col, target_col, seed = 0):
        """
        Se cargan los targets y los samples recibidos y tambien se los mezcla. Ademas, se genera un diccionario para poder codificar
        y decodificar los targets.
        @Args:
            df: Data frame de pandas en el cual se tienen los datos del dataSet
            path_col: String asociado a la columna de df con el path donde se encuentran los archivos de las samples
            file_col: String asociado a la columna de df que contiene los nombres de los archivos de samples
            target_col: String asociado a la columna de df que tiene las categorias de los targets
            seed: (opcional) Si este paramatro es recibido, se fija la semilla para el randomizado del dataSet 
        """
        if seed != 0:
            np.random.seed(seed) #fijo la semilla para repetibilidad del proyecto
        df.sample(frac = 1) #cargo y mezlco el dataSet
        self.dictionary = self._gen_dictionary(df[target_col])
        self.targets = self.code_batch(df[target_col])
        self.l = len(df)

        fPaths = '.' + df[path_col].str.strip() + '/' + df[file_col]
        self.mels = self._load_files(fPaths)          #lista de espectrogramas
        return    
    def __len__(self):
        return self.l

    def __getitem__(self, index):
        """
        @Args:
        index: indice del elemento a obtener
        @Returns:
        Samlpe: Espectrograma de Mel asociado al indice
        Target: Target asociado al indice
        """
        return self.mels[index], self.targets[index]       
    
    def _load_files(self, files):
        """
        @Args:
            files: lista de archivos en formato pt
        @Return:
            mels: lista con los datos cargados
        """
        count = 0
        mels = []
        for file in files:
            mels.append(torch.load(file))

            count = count + 1
            if (not (count % 100)):
                print(f"Cargados: {count}/{self.l}")
        return mels

    def _gen_dictionary(self, col):
        """
        Genera un diccionario cullas claves son los valores posibles de un data fram columna recibido
        @Args:
            col: dataFrame columna.
        @Return:
            diccionario: diccionario que transforma cada valor de la columna a un formato oneHot encoing
        """
        #obtengo los valores positbles de la columna objetivo
        keys = col.unique()
        #Genero un diccionario que transforma las keys en one hot
        key_to_oneHot = {}
        for i in range(len(keys)):
            one_hot_vector = [0] * len(keys)
            one_hot_vector[i] = 1
            key_to_oneHot [keys[i]] = one_hot_vector
        return key_to_oneHot 
   
    def code_batch(self, values):
        """
        @Args:
            values: Lista de entradas a codificar. (se puede decodicifca con la funcion decode)
        @Returns:
            coded: Lista con las entras codificadas en formato one hot encoding. Ejemplo: [[0,0,1], [0,1,0]]
        """
        coded = [self.dictionary[value] for value in values]
        return coded
    
    def code(self, value):
        """
        Codifica value a one hot ecoding (se recupera con el meotod decode)
        """
        return self.dictionary[value]
            
    def decode(self, value):
        """
        Se recibe un vector codificado en formato one hot y un diccionario el cual se uso para codificar el one hot
        De modo qu ese retorna la key <string> decodificada del vector recibido.
        @Args:
            value: vector codificado en oneHot que se quiere recuperar
        @Returns:
            restored: string con la palabra asociada al vector oneHot recibido.
        """
        restored = 'NotFound'
        for key,dic_value in self.dictionary.items():
            if value == dic_value:
                restored = key
                break
        return restored
#test

In [154]:
testDf = pd.read_csv('./csv/test_set.csv')
testDataSet = MelDataset(testDf, "localPath", "fileMel", "M/F/I", seed = 98)

Cargados: 100/462
Cargados: 200/462
Cargados: 300/462
Cargados: 400/462


In [155]:
trainDf = pd.read_csv('./csv/train_set.csv')
testDataSet = MelDataset(trainDf, "localPath", "fileMel", "M/F/I", seed = 98)

Cargados: 100/13872
Cargados: 200/13872
Cargados: 300/13872
Cargados: 400/13872
Cargados: 500/13872
Cargados: 600/13872
Cargados: 700/13872
Cargados: 800/13872
Cargados: 900/13872
Cargados: 1000/13872
Cargados: 1100/13872
Cargados: 1200/13872
Cargados: 1300/13872
Cargados: 1400/13872
Cargados: 1500/13872
Cargados: 1600/13872
Cargados: 1700/13872
Cargados: 1800/13872
Cargados: 1900/13872
Cargados: 2000/13872
Cargados: 2100/13872
Cargados: 2200/13872
Cargados: 2300/13872
Cargados: 2400/13872
Cargados: 2500/13872
Cargados: 2600/13872
Cargados: 2700/13872
Cargados: 2800/13872
Cargados: 2900/13872
Cargados: 3000/13872
Cargados: 3100/13872
Cargados: 3200/13872
Cargados: 3300/13872
Cargados: 3400/13872
Cargados: 3500/13872
Cargados: 3600/13872
Cargados: 3700/13872
Cargados: 3800/13872
Cargados: 3900/13872
Cargados: 4000/13872
Cargados: 4100/13872
Cargados: 4200/13872
Cargados: 4300/13872
Cargados: 4400/13872
Cargados: 4500/13872
Cargados: 4600/13872
Cargados: 4700/13872
Cargados: 4800/13872
C

In [112]:
class Peperino():
    def __init__(self, name):
        self.name = self.foo(name)
        self.hello()

    def foo(self,name):
        return name*2
    def hello(self):
        print(f"Hello my name is {self.name}")

# Cargando todos los resultados posibles

In [ ]:
allMels = []
count = 0
for path in fPaths.iloc:
    allMels.append( torch.load(path))
    count = count + 1
    if(not count%1000):
        print(count/len(allMels))